## Phase 1: Inference of co-expression modules 
### Run GENIE3 with random forest, without randomness (K=p-1, p number of genes)
* in https://pubmed.ncbi.nlm.nih.gov/20927193/ this way produced best results


In [ ]:
library(GENIE3)